# Improvements on RQAE algorithms

Several modifications over the **RQAE** were developed in the **QQuantLib** library. These modifications can be found inside the **QQuantLib.AE** package in the following modules and clasess:

* Modified **RQAE** in the **mRQAE** class inside the *modified_real_quantum_ae* module.
* Shots version of the **RQAE** in the **sRQAE** class inside the *shots_real_quantum_ae* module.
* Extended **RQAE** in the **eRQAE** class inside the *extended_real_quantum_ae* module.


All these modifications use the **RQAE** quantum circuits shown in section 3 of jupyter notebook: *07_Real_Quantum_Amplitude_Estimation_class*. The main difference is how the classical part is configured in each variation. Playing with the classical part the performance of the **RQAE** algorithm can be improved a lot and even can achieved better performances than other **AE** algorithms like **IQAE** or **mIQAE**.

All these implementations work in the same way than the original **RQAE** class

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver. See notebook: 00_AboutTheNotebooksAndQPUs.ipynb
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus
myqlm_qpus = ["python", "c"]
# QLM qpus accessed using Qaptiva Access library
qlmass_qpus = ["qlmass_linalg", "qlmass_mps"]
# QLM qpus: Only in local Quantum Learning Machine
qlm_qpus = ["linalg", "mps"]

linalg_qpu = get_qpu(myqlm_qpus[1])#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results
from QQuantLib.utils.utils import bitfield_to_int

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case, we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability* from **QQuantLib/DL/data_loading** module. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from QQuantLib.DL.data_loading import load_probability
# For comparing RQAE modifications

n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

#First we create the class
target = [0,0,1]
index = [0,1,2]
a = np.sqrt(probability[bitfield_to_int(target)])
print('We want to estimate: ', a)

## 2. RQAE original algorithm

For comparison purpouses we are going to execute the original RQAE algorithm

In [ ]:
from QQuantLib.AE.real_quantum_ae import RQAE

In [ ]:
epsilon = 0.001
q = 2
gamma = 0.05

In [ ]:
rqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'mcz_qlm': False
}
rqae_ = RQAE(oracle, target = target, index = [0,1,2], **rqae_dict)
_ = rqae_.run()
print('a_estimated: {}. RQAE: {}.'.format(a, rqae_.ae))
print('Errors: RQAE: {}.'.format(np.abs(a - rqae_.ae)))
print("Oracle calls: RQAE: {}.".format(rqae_.oracle_calls))

As explained in *07_Real_Quantum_Amplitude_Estimation_class.ipynb* notebook *display_information* provides info abput the asymptotic bounds of the RQAE for a given $\epsilon$, $\gamma$ and $q$

In [ ]:
rqae_.display_information(ratio=q, epsilon=epsilon, gamma=gamma)

## 3. Modified RQAE (mRQAE)


In the original **RQAE** algorithm the failure probability at each step of the algorithm, $\gamma_i$, is kept constant. In the **mRQAE** this failure probability (and the corresponding number of shots) changes with the amplification of the step. It can be show that setting this probability to 

$$\gamma_i =\frac{0.5 * \gamma * (q - 1) * (2k + 1)}{q * (2 * k_{max} + 1)}$$ 

where $q$ is the ratio, $k$ the number of grover operators to apply, $\gamma$ the final probability failure desired and $k_{max}$ the maximum number of times the Grover operator will be applied (depends on the desired $\epsilon$) the asymptotyc query behaviuor is improved over the **RQAE** one:

* RQAE query complexity: $\sim \frac{1}{\epsilon} \log \left( \frac{1}{\alpha} \log \left(\frac{1}{\epsilon}\right)\right)$
* mRQAE query complexity: $\sim \frac{1} {\epsilon} \log \frac{1}{\alpha}$

In [ ]:
from QQuantLib.AE.modified_real_quantum_ae import mRQAE

mrqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'mcz_qlm': False
}

mrqae_ = mRQAE(oracle, target = target, index = [0,1,2], **mrqae_dict)
_ = mrqae_.run()


print('a_estimated: {}. RQAE: {}. mRQAE: {}'.format(a, rqae_.ae, mrqae_.ae))
print('Errors: RQAE: {}. mRQAE: {}'.format(
    np.abs(a - rqae_.ae), np.abs(mrqae_.ae-a))
)
print("Oracle calls: RQAE: {}. mRQAE: {}".format(rqae_.oracle_calls, mrqae_.oracle_calls))

We can compare the bounds for both methods by calling the method *compute_info* that provides info about the bounds onf the algorithm

In [ ]:
eps_list= np.logspace(-1, -11)
rqae_grover = [rqae_.compute_info(ratio=q, epsilon=x, gamma=gamma)["n_oracle"] for x in eps_list]
mrqae_grover = [mrqae_.compute_info(ratio=q, epsilon=x, gamma=gamma)["n_oracle"] for x in eps_list]

plt.plot(eps_list, rqae_grover)
plt.plot(eps_list, mrqae_grover)
plt.xscale("log")
plt.yscale("log")
xmin, xmax, ymin, ymax = plt.axis()
plt.xlim(xmax, xmin)
plt.legend(["RQAE", "mRQAE"])
plt.xlabel(r"$\epsilon$")
plt.ylabel(r"Oracle Calls")

__Experimentally the *mRQAE* has a better performance than the original *RQAE* (lower oracle call for same desired epsilons)__

The *display_information* summarizes the asymptotic bounds for a **mRQAE** given an input: $\epsilon$, $\gamma$ and $q$

In [ ]:
mrqae_.display_information(ratio=q, epsilon=epsilon, gamma=gamma)

## 4. RQAE with shots

In the original **RQAE** the number of shots for each iteration is fixed internally in the algorithm. In the class **sRQAE** from the module **QQuantLib.AE.shots_real_quantum_ae** a **RQAE** version where the user can provide the number of shots as inputs (like in the **IQAE** and **mIQAE** algorithms) was developed. 

By providing an input number of shots to the **sRQAE** algortihm better performances can be achieved experimentally (lower calls to the oracle for the same desired $\epsilon$, $\gamma$ and $q$) compared with **RQAE** and even with **mRQAE**

In [ ]:
from QQuantLib.AE.shots_real_quantum_ae import sRQAE

In [ ]:
srqae_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

srqae_ = sRQAE(oracle, target = target, index = [0,1,2], **srqae_dict)
_ = srqae_.run()

In [ ]:
print('a_estimated: {}. RQAE: {}. mRQAE: {}. sRQAE'.format(a, rqae_.ae, mrqae_.ae, srqae_.ae))
print('Errors: RQAE: {}. mRQAE: {}. sRQAE: {}'.format(
    np.abs(a - rqae_.ae), np.abs(mrqae_.ae-a), np.abs(srqae_.ae-a)
)
)
print("Oracle calls: RQAE: {}. mRQAE: {}. sRQAE: {}".format(
    rqae_.oracle_calls, mrqae_.oracle_calls, srqae_.oracle_calls
))

**BE AWARE**

In **RQAE** with shots the asymptotic query behaviour depends on number of shots so we do not provide any information about them. So *display_information* do not provide any value

In [ ]:
srqae_.display_information(epsilon=epsilon, gamma=gamma, ratio = q)

## 5. Extended RQAE.

Last modification provided for **RQAE** algorithm is the *extended RQAE* algorithm implemented in the class **eRQAE** of the module **QQuantLib.AE.extended_real_quantum_ae**.  In this case the user can guide the evolution of the number of Grover applications ($k$) and the failure probabiliy ($\gamma_i$) at each step of the algorithm. For guiding this evolution **eRQAE** uses 2 list (schedules) of the same lenght: one for guiding the evolution of $k$ and another for the evolution of $\gamma_i$.

For helping the user to design these schedules four different functions in the **QQuantLib.AE.extended_real_quantum_ae** module were developed:
1. *schedule_exponential_constant*
2. *schedule_exponential_exponential*
3. *schedule_linear_linear*
4. *schedule_linear_constant*

The **eRQAE** class uses these functions for creating the used schedules. The user can select the different functions and their parameters by providing to the **eRQAE** class the keyword argument *erqae_schedule*. This *erqae_schedule* is a Python dictionary with the following format:

**{"type": type, "ratio_slope_k": ratio_slope_k, "ratio_slope_gamma": ratio_slope_gamma}**

Where:

* type: a string that indicates the scheduling function to use:
    * "exp_const" for *schedule_exponential_constant* function.
    * "exp_exp" for *schedule_exponential_exponential* function.
    * "linear_linear" for *schedule_linear_linear* function.
    * "linear_const" for *schedule_linear_constant* function.
* ratio_slope_k: ratio or slope for $k$ schedule
* ratio_slope_gamma: ratio or slope for $\gamma$ schedule.


Under the hood the **eRQAE** class call to a select function called **select_schedule** (in the **QQuantLib.AE.extended_real_quantum_ae** module) that acts as a selector function of the different scheduling functions. The inputs of the **select_schedule** are:
* *erqae_schedule*: python dictionary with the same format that the keyword argument: **erqae_schedule**
* epsilon: the desired $\epsilon$ to achieve.
* gamma: the desired $\gamma$ to achieve.

In the following subsections we explain the different scheduling functions and how to configure them.

In [ ]:
from QQuantLib.AE.extended_real_quantum_ae import select_schedule, eRQAE

### 5.1 schedule_exponential_constant

In this case we want a exponential evolution (schedule) for the $k$ and a constant one for the failure probability $\gamma_i$. In this case the input *erqae_schedule* dictionary will have the following format:
* type: exp_const
* ratio_slope_k: desired ratio
* ratio_slope_gamma: None.

We can use the **select_schedule** for getting the obtained schedules:


In [ ]:
# the erqae_schedule dictionary
exp_k_const_gamma = {
    "type" : "exp_const",
    "ratio_slope_k": 3.5,
    "ratio_slope_gamma": None
}

k_exp, gamma_cte = select_schedule(exp_k_const_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_exp, 'bo-')
ax2.plot(gamma_cte, 'ro-')
ax1.set_ylabel('k schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
erqae_exp_k_const_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': exp_k_const_gamma,
    'mcz_qlm': False
}

erqae_k_exp_gamma_cte = eRQAE(oracle, target = target, index = [0,1,2], **erqae_exp_k_const_gamma_dict)
_ = erqae_k_exp_gamma_cte.run()

print("### Schedule: exponential in k constant in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_k_exp_gamma_cte.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_k_exp_gamma_cte.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_k_exp_gamma_cte.oracle_calls,
))

### 5.2 schedule_exponential_exponential 

In this case we want a exponential schedule for the $k$ and for the failure probability $\gamma_i$. In this case the input *erqae_schedule* dictionary will have the following format:
* type: exp_exp
* ratio_slope_k: desired ratio for k
* ratio_slope_gamma: desired ratio for gamma

**BE AWARE**
The ratio for $\gamma$ can be positive or negative:
* When ratio_slope_gamma > 0: lower probability failures at the initial steps. The probability failure is increasing with the different steps
* When ratio_slope_gamma < 0: higher probability failures at the initial steps. The probability failure is decreasing with the different steps

We can use the **select_schedule** for getting the obtained schedules:


In [ ]:
# the erqae_schedule dictionary
exp_k_exp_gamma = {
    "type" : "exp_exp",
    "ratio_slope_k": 2.5,
    "ratio_slope_gamma": 3.5
}

k_exp, gamma_exp = select_schedule(exp_k_exp_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_exp, 'bo-')
ax2.plot(gamma_exp, 'ro-')
ax1.set_ylabel('K schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
exp_k_exp_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': exp_k_exp_gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

erqae_exp_k_exp_gamma = eRQAE(oracle, target = target, index = [0,1,2], **exp_k_exp_gamma_dict)
_ = erqae_exp_k_exp_gamma.run()

print("### Schedule: exponential in k exponential in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_exp_k_exp_gamma.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_exp_k_exp_gamma.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_exp_k_exp_gamma.oracle_calls,
))

In [ ]:
#Neagtive ratio fo gamma
# the erqae_schedule dictionary
exp_k_exp_gamma = {
    "type" : "exp_exp",
    "ratio_slope_k": 2.5,
    "ratio_slope_gamma": -3.5
}

k_exp, gamma_exp = select_schedule(exp_k_exp_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_exp, 'bo-')
ax2.plot(gamma_exp, 'ro-')
ax1.set_ylabel('K schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
exp_k_exp_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': exp_k_exp_gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

erqae_exp_k_exp_gamma = eRQAE(oracle, target = target, index = [0,1,2], **exp_k_exp_gamma_dict)
_ = erqae_exp_k_exp_gamma.run()

print("### Schedule: exponential in k exponential in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_exp_k_exp_gamma.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_exp_k_exp_gamma.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_exp_k_exp_gamma.oracle_calls,
))

### 5.3 schedule_linear_linear 

In this case we want a linear schedule for the $k$ and for the failure probability $\gamma_i$. In this case the input *erqae_schedule* dictionary will have the following format:
* type: linear_linear
* ratio_slope_k: desired slope for k
* ratio_slope_gamma: desired slope for gamma

**BE AWARE**
The slope for $\gamma$ can be positive or negative:
* When ratio_slope_gamma > 0: lower probability failures at the initial steps. The probability failure is increasing with the different steps
* When ratio_slope_gamma < 0: higher probability failures at the initial steps. The probability failure is decreasing with the different steps

We can use the **select_schedule** for getting the obtained schedules:


In [ ]:
# the erqae_schedule dictionary
linear_k_linear_gamma = {
    "type" : "linear_linear",
    "ratio_slope_k": 2.5,
    "ratio_slope_gamma": 4.5
}

k_linear, gamma_linear = select_schedule(linear_k_linear_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_linear, 'bo-')
ax2.plot(gamma_linear, 'ro-')
ax1.set_ylabel('K schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
linear_k_linear_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': linear_k_linear_gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

erqae_linear_k_linear_gamma = eRQAE(oracle, target = target, index = [0,1,2], **linear_k_linear_gamma_dict)
_ = erqae_linear_k_linear_gamma.run()


print("### Schedule: linear in k linear in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_linear_k_linear_gamma.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_linear_k_linear_gamma.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_linear_k_linear_gamma.oracle_calls,
))

In [ ]:
#Negative gamma slope
# the erqae_schedule dictionary
linear_k_linear_gamma = {
    "type" : "linear_linear",
    "ratio_slope_k": 2.5,
    "ratio_slope_gamma": -4.5
}

k_linear, gamma_linear = select_schedule(linear_k_linear_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_linear, 'bo-')
ax2.plot(gamma_linear, 'ro-')
ax1.set_ylabel('K schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
linear_k_linear_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': linear_k_linear_gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

erqae_linear_k_linear_gamma = eRQAE(oracle, target = target, index = [0,1,2], **linear_k_linear_gamma_dict)
_ = erqae_linear_k_linear_gamma.run()


print("### Schedule: linear in k linear in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_linear_k_linear_gamma.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_linear_k_linear_gamma.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_linear_k_linear_gamma.oracle_calls,
))

### 5.4 schedule_linear_constant 

In this case we want a linear schedule for the $k$ and a constant one for the failure probability $\gamma_i$. In this case the input *erqae_schedule* dictionary will have the following format:
* type: linear_const
* ratio_slope_k: desired slope for k
* ratio_slope_gamma: None


We can use the **select_schedule** for getting the obtained schedules:


In [ ]:
#Negative gamma slope
# the erqae_schedule dictionary
linear_k_cte_gamma = {
    "type" : "linear_const",
    "ratio_slope_k": 2.5,
    "ratio_slope_gamma": None
}

k_linear, gamma_cte = select_schedule(linear_k_cte_gamma, epsilon=epsilon, gamma=gamma)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(k_linear, 'bo-')
ax2.plot(gamma_cte, 'ro-')
ax1.set_ylabel('K schedule', color='b')
ax2.set_ylabel(r'$\gamma$ scshdule', color='r')

In [ ]:
# we use the before erqae_schedule dictioanry
linear_k_cte_gamma_dict = {
    'qpu': linalg_qpu,    
    'epsilon': epsilon,
    'ratio': q,
    'gamma': gamma,
    'erqae_schedule': linear_k_cte_gamma,
    'shots': 100, # Now we provide shots!!
    'mcz_qlm': False
}

erqae_linear_k_cte_gamma = eRQAE(oracle, target = target, index = [0,1,2], **linear_k_cte_gamma_dict)
_ = erqae_linear_k_cte_gamma.run()


print("### Schedule: linear in k constant in gamma ####")
print('a_estimated: {}. RQAE: {}. eRQAE: {}'.format(a, rqae_.ae, erqae_linear_k_cte_gamma.ae))
print('Errors: RQAE: {}. eRQAE: {}.'.format(
    np.abs(a - rqae_.ae), np.abs(erqae_linear_k_cte_gamma.ae-a),
)
)
print("Oracle calls: RQAE: {}. eRQAE: {}.".format(
    rqae_.oracle_calls, erqae_linear_k_cte_gamma.oracle_calls,
))

**BE AWARE**

In **eRQAE** the asymptotic query behaviour depends on the selected schedule so we do not provide any information about them. So *display_information* do not provide any value

In [ ]:
erqae_linear_k_cte_gamma.display_information(epsilon=epsilon, gamma=gamma, ratio = q)